In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from collections import Counter
from datetime import datetime

In [5]:
df_scores = pd.read_csv("compas-scores.csv")
df_scores = df_scores[df_scores.c_charge_degree != 'O']

In [53]:
df_scores.head(n=5)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13
5,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-03-26,Risk of Recidivism,1,Low,2013-03-26


In [7]:
battery = ['batt']
assault = ['assault', 'abuse']
drugs = ['subst', 'cannabi', 'cocaine', 'deliver', 'del ', 'alprazolam', 'pyrro', 'heroin', 
              'hydro','ethylone', 'drug', 'anabolic', 'steroid',
             'tamine', 'meth', 'codeine', 'lsd',
             'buprenorphine', 'lorazepam','carisoprodol', 'benzyl',
             'clonazepam', 'diazepam', 'morph', 'fluor', ',4', 'fentanyl',
             'phentermine', 'butylone']
driving = ['dui', 'driv', 'veh', 'operat', ' dl', 'd.u.i', 'dwi']
prop = ['burg', 'arson', 'theft', 'property', 'mischief', 'trespass', 'fraud', 'stole']
weapon = ['firearm', 'weapon', 'arm', 'wep']
sexual = ['sex', 'prostitu', 'lewd', 'voyeur', 'porn']
obstruct = ['flee', 'forg', 'tamper', 'false', 'obstruct', 'obey', 'resist', ' id']
domestic = ['dom', 'child']
agg = ['agg']
violent = ['rape', 'assault', 'abuse', 'kidnap', 'murder', 'manslaughter', 'batt', 'assault', 'abuse']
public_order = ['weapon', 'firearm', 'arm', 'dui', 'driv', 'd.u.i.', 'dwi']

["('Prostitution/Lewd Act Assignation', 14)", "('Lewd or Lascivious Molestation', 12)", "('Felony Committing Prostitution', 6)", "('Lewd/Lasc Battery Pers 12+/<16', 5)", "('Possession Child Pornography', 5)", "('Live on Earnings of Prostitute', 5)", "('Sexual Battery / Vict 12 Yrs +', 4)", "('Sexual Performance by a Child', 4)", "('Soliciting For Prostitution', 2)", "('Prostitution/Lewdness/Assign', 2)", "('Offer Agree Secure/Lewd Act', 2)", "('Sex Battery Deft 18+/Vict 11-', 2)", "('Lewdness Violation', 2)", "('Fail to Reg as Sexual Offender', 2)", "('Sex Offender Fail Comply W/Law', 2)", "('Voyeurism', 1)", "('Offer Agree Secure For Lewd Act', 1)", "('Sex Batt Faml/Cust Vict 12-17Y', 1)", "('Procure For Prostitution', 1)", "('Solicit ProstitutionViolation', 1)", "('Aide/Abet Prostitution Lewdness', 1)", "('Lewd Act Presence Child 16-', 1)", "('Lewd/Lasciv Molest Elder Persn', 1)", "('Video Voyeur-<24Y on Child >16', 1)", "('Unlawful Sexual Activity', 1)", "('Fail Sex Offend Report By

In [34]:
df_full = pd.read_csv('compas-features-nobinary-id.csv')
df_train, df_test = train_test_split(df_full, test_size=0.33)
df_train.to_csv('compas-features-nobinary-id-train.csv', index=False)
df_test.to_csv('compas-features-nobinary-id-test.csv', index=False)

In [55]:
charges = df_scores['c_charge_desc'].unique()

no_drug_crimes = []
traff = []
for charge in charges:
    if not (any(d in str(charge).lower() for d in drugs)):
        no_drug_crimes.append(charge)

len(no_drug_crimes)

419

In [134]:
index = 0
c_drugs_count = 0
r_drugs_count = 0
drug_entries = np.zeros((df_scores.shape[0]))
INDEX = list(df_scores.columns).index('r_charge_desc')
INDEX_id = list(df_scores.columns).index('id')
for i in range(len(df_scores['r_charge_desc'])):
#for i in range(20):
    recid = str(df_scores.iloc[i, INDEX]).lower()
    if any(d in recid for d in drugs):
        drug_entries[index] = df_scores.iloc[i, INDEX_id]
        r_drugs_count += 1
        index +=1
    
    #crime = str(df_scores.iloc[i, 'c_charge_desc']).lower()
    #if any(d in crime for d in drugs):
        #c_drugs_count += 1

print c_drugs_count, r_drugs_count

0 691


In [135]:
df_full = pd.read_csv('compas-features-nobinary-id.csv')
df_full.head(n=5)

,is_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
3,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5
4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,0,0,0,43,6


In [61]:
df_full.loc[df_full['id'] == 24]

,is_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
22,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5,0,0,0,31,24


In [136]:
df_joined_d = df_full.loc[df_full['id'] == int(drug_entries[0])]
#df_joined['is_drug_recid'] = pd.Series(np.zeros(1), index=df_joined.index)
df_joined_d = df_joined_d.assign(is_drug_recid=pd.Series(np.zeros(1), index=df_joined_d.index))

INDEX_id = list(df_joined_d.columns).index('id')
INDEX_drug_recid = list(df_joined_d.columns).index('is_drug_recid')
if int(df_joined_d.iloc[0, INDEX]) in drug_entries:
    df_joined_d.iloc[0, INDEX_drug_recid] = 1
else:
    df_joined_d.iloc[0, INDEX_drug_recid] = 0

for i in df_full['id']:
    if i == drug_entries[0]:
        print 'equal'
        continue
    df_temp = df_full.loc[df_full['id'] == int(i)]
    df_temp = df_temp.assign(is_drug_recid=pd.Series(np.zeros(1), index=df_temp.index))
    if i in drug_entries:
        df_temp.iloc[0, INDEX_drug_recid] = 1
    else:
        df_temp.iloc[0, INDEX_drug_recid] = 0
    df_joined_d = pd.concat([df_joined_d, df_temp])

equal


In [137]:
cols = list(df_joined_d.columns)
INDEX = cols.index('is_drug_recid')
new_cols = []
new_cols.append(cols[INDEX])
for i in range(len(cols)):
    if i != INDEX:
        new_cols.append(cols[i])
new_cols.remove('is_recid')
df_joined_reorder_d = df_joined_d[new_cols]
df_joined_reorder_d.head()

,is_drug_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
22,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5,0,0,0,31,24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5


In [64]:
df_joined.shape

(11003, 45)

In [138]:
df_joined_reorder_d.to_csv('compas-r-drugs.csv', index=False)

In [139]:
#nrows = df_joined_reorder.shape[0]
#df_joined_reorder.sample(n=int(.7*nrows))
#x.head(n=5)
train_d, test_d = train_test_split(df_joined_reorder_d, test_size = 0.33) 
train_d.to_csv('compas-r-drugs-train.csv', index=False)
test_d.to_csv('compas-r-drugs-test.csv', index=False)

In [140]:
df_joined_reorder_d[df_joined_reorder_d.is_drug_recid == 1].shape

(691, 44)

# Sexual

In [8]:
index = 0
r_sexual_count = 0
sexual_entries = np.zeros((df_scores.shape[0]))
INDEX = list(df_scores.columns).index('r_charge_desc')
INDEX_id = list(df_scores.columns).index('id')
for i in range(len(df_scores['r_charge_desc'])):
#for i in range(20):
    recid = str(df_scores.iloc[i, INDEX]).lower()
    if any(d in recid for d in sexual):
        sexual_entries[index] = df_scores.iloc[i, INDEX_id]
        r_sexual_count += 1
        index +=1

print r_sexual_count

37


In [9]:
df_full = pd.read_csv('compas-features-nobinary-id.csv')
df_full.head(n=5)

,is_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
3,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5
4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,0,0,0,43,6


In [10]:
24 in sexual_entries[:38]

False

In [11]:
df_joined_s = df_full.loc[df_full['id'] == int(sexual_entries[0])]
df_joined_s = df_joined_s.assign(is_sexual_recid=pd.Series(np.zeros(1), index=df_joined_s.index))

INDEX_id = list(df_joined_s.columns).index('id')
INDEX_sexual_recid = list(df_joined_s.columns).index('is_sexual_recid')
if int(df_joined_s.iloc[0, INDEX]) in sexual_entries:
    df_joined_s.iloc[0, INDEX_sexual_recid] = 1
else:
    df_joined_s.iloc[0, INDEX_sexual_recid] = 0

for i in df_full['id']:
    if i == sexual_entries[0]:
        print 'equal'
        continue
    df_temp = df_full.loc[df_full['id'] == int(i)]
    df_temp = df_temp.assign(is_sexual_recid=pd.Series(np.zeros(1), index=df_temp.index))
    if i in sexual_entries:
        df_temp.iloc[0, INDEX_sexual_recid] = 1
    else:
        df_temp.iloc[0, INDEX_sexual_recid] = 0
    df_joined_s = pd.concat([df_joined_s, df_temp])

equal


In [12]:
cols = list(df_joined_s.columns)
INDEX = cols.index('is_sexual_recid')
new_cols = []
new_cols.append(cols[INDEX])
for i in range(len(cols)):
    if i != INDEX:
        new_cols.append(cols[i])
new_cols.remove('is_recid')
df_joined_reorder_s = df_joined_s[new_cols]
df_joined_reorder_s.head()

,is_sexual_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
217,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7,0,0,0,33,228
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5


In [13]:
print df_joined_reorder_s.shape
df_joined_reorder_s.to_csv('compas-r-sexual.csv', index=False)
df_joined_reorder_s.head()

(11003, 44)


,is_sexual_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
217,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7,0,0,0,33,228
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5


In [15]:
train_d, test_d = train_test_split(df_joined_reorder_s, test_size = 0.33) 
train_d.to_csv('compas-r-sexual-train.csv', index=False)
test_d.to_csv('compas-r-sexual-test.csv', index=False)

# Violent

In [16]:
index = 0
r_violent_count = 0
violent_entries = np.zeros((df_scores.shape[0]))
INDEX = list(df_scores.columns).index('is_violent_recid')
INDEX_id = list(df_scores.columns).index('id')
for i in range(len(df_scores['r_charge_desc'])):
#for i in range(20):
    v_recid = int(df_scores.iloc[i, INDEX])
    #if any(d in recid for d in agg) or any(d in recid for d in property):
    if v_recid == 1:
        violent_entries[index] = df_scores.iloc[i, INDEX_id]
        r_violent_count += 1
        index +=1

print r_violent_count

877


In [17]:
df_joined_v = df_full.loc[df_full['id'] == int(violent_entries[0])]
df_joined_v = df_joined_v.assign(is_violent_recid=pd.Series(np.zeros(1), index=df_joined_v.index))

INDEX_id = list(df_joined_v.columns).index('id')
INDEX_violent_recid = list(df_joined_v.columns).index('is_violent_recid')
if int(df_joined_v.iloc[0, INDEX]) in violent_entries:
    df_joined_v.iloc[0, INDEX_violent_recid] = 1
else:
    df_joined_v.iloc[0, INDEX_violent_recid] = 0

for i in df_full['id']:
    if i == violent_entries[0]:
        print 'equal'
        continue
    df_temp = df_full.loc[df_full['id'] == int(i)]
    df_temp = df_temp.assign(is_violent_recid=pd.Series(np.zeros(1), index=df_temp.index))
    if i in violent_entries:
        df_temp.iloc[0, INDEX_violent_recid] = 1
    else:
        df_temp.iloc[0, INDEX_violent_recid] = 0
    df_joined_v = pd.concat([df_joined_v, df_temp])

equal


In [18]:
cols = list(df_joined_v.columns)
INDEX = cols.index('is_violent_recid')
new_cols = []
new_cols.append(cols[INDEX])
for i in range(len(cols)):
    if i != INDEX:
        new_cols.append(cols[i])
new_cols.remove('is_recid')
df_joined_reorder_v = df_joined_v[new_cols]
df_joined_reorder_v.head()

,is_violent_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,0,0,0,43,6


In [19]:
#(df_joined_reorder[df_joined_reorder.is_violent_recid == 1.0]).shape

In [20]:
print df_joined_reorder_v.shape
df_joined_reorder_v.to_csv('compas-r-violent.csv', index=False)
train_v, test_v = train_test_split(df_joined_reorder_v, test_size = 0.33) 
train_v.to_csv('compas-r-violent-train.csv', index=False)
test_v.to_csv('compas-r-violent-test.csv', index=False)

(11003, 44)


# Property crime

In [21]:
index = 0
r_prop_count = 0
prop_entries = np.zeros((df_scores.shape[0]))
INDEX = list(df_scores.columns).index('r_charge_desc')
INDEX_id = list(df_scores.columns).index('id')
for i in range(len(df_scores['r_charge_desc'])):
    recid = str(df_scores.iloc[i, INDEX]).lower()
    if any(d in recid for d in prop):
        prop_entries[index] = df_scores.iloc[i, INDEX_id]
        r_prop_count += 1
        index +=1

print r_prop_count

772


In [22]:
df_joined_pr = df_full.loc[df_full['id'] == int(prop_entries[0])]
df_joined_pr = df_joined_pr.assign(is_prop_recid=pd.Series(np.zeros(1), index=df_joined_pr.index))

INDEX_id = list(df_joined_pr.columns).index('id')
INDEX_prop_recid = list(df_joined_pr.columns).index('is_prop_recid')
if int(df_joined_pr.iloc[0, INDEX]) in prop_entries:
    df_joined_pr.iloc[0, INDEX_prop_recid] = 1
else:
    df_joined_pr.iloc[0, INDEX_prop_recid] = 0

for i in df_full['id']:
    if i == prop_entries[0]:
        print 'equal'
        continue
    df_temp = df_full.loc[df_full['id'] == int(i)]
    df_temp = df_temp.assign(is_prop_recid=pd.Series(np.zeros(1), index=df_temp.index))
    if i in prop_entries:
        df_temp.iloc[0, INDEX_prop_recid] = 1
    else:
        df_temp.iloc[0, INDEX_prop_recid] = 0
    df_joined_pr = pd.concat([df_joined_pr, df_temp])

equal


In [23]:
cols = list(df_joined_pr.columns)
INDEX = cols.index('is_prop_recid')
new_cols = []
new_cols.append(cols[INDEX])
for i in range(len(cols)):
    if i != INDEX:
        new_cols.append(cols[i])
new_cols.remove('is_recid')
df_joined_reorder_pr = df_joined_pr[new_cols]
df_joined_reorder_pr.head()

,is_prop_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,0,0,47,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5


In [24]:
print df_joined_reorder_pr.shape
df_joined_reorder_pr.to_csv('compas-r-property.csv', index=False)
train_pr, test_pr = train_test_split(df_joined_reorder_pr, test_size = 0.33) 
train_pr.to_csv('compas-r-property-train.csv', index=False)
test_pr.to_csv('compas-r-property-test.csv', index=False)

(11003, 44)


# Public Order

In [25]:
index = 0
c_po_count = 0
r_po_count = 0
po_entries = np.zeros((df_scores.shape[0]))
INDEX = list(df_scores.columns).index('r_charge_desc')
INDEX_id = list(df_scores.columns).index('id')
for i in range(len(df_scores['r_charge_desc'])):
    recid = str(df_scores.iloc[i, INDEX]).lower()
    if any(d in recid for d in public_order):
        po_entries[index] = df_scores.iloc[i, INDEX_id]
        r_po_count += 1
        index +=1
    
    #crime = str(df_scores.iloc[i, 'c_charge_desc']).lower()
    #if any(d in crime for d in drugs):
        #c_drugs_count += 1

print c_po_count, r_po_count

0 625


In [26]:
df_joined_po = df_full.loc[df_full['id'] == int(po_entries[0])]
df_joined_po = df_joined_po.assign(is_po_recid=pd.Series(np.zeros(1), index=df_joined_po.index))

INDEX_id = list(df_joined_po.columns).index('id')
INDEX_po_recid = list(df_joined_po.columns).index('is_po_recid')
if int(df_joined_po.iloc[0, INDEX]) in po_entries:
    df_joined_po.iloc[0, INDEX_po_recid] = 1
else:
    df_joined_po.iloc[0, INDEX_po_recid] = 0

for i in df_full['id']:
    if i == po_entries[0]:
        print 'equal'
        continue
    df_temp = df_full.loc[df_full['id'] == int(i)]
    df_temp = df_temp.assign(is_po_recid=pd.Series(np.zeros(1), index=df_temp.index))
    if i in po_entries:
        df_temp.iloc[0, INDEX_po_recid] = 1
    else:
        df_temp.iloc[0, INDEX_po_recid] = 0
    df_joined_po = pd.concat([df_joined_po, df_temp])

equal


In [27]:
cols = list(df_joined_po.columns)
INDEX = cols.index('is_po_recid')
new_cols = []
new_cols.append(cols[INDEX])
for i in range(len(cols)):
    if i != INDEX:
        new_cols.append(cols[i])
new_cols.remove('is_recid')
df_joined_reorder_po = df_joined_po[new_cols]
df_joined_reorder_po.head()

,is_po_recid,age_less_18,age_19_24,age_25_29,age_30_34,age_35_39,age_40_44,age_45_49,age_50_54,age_55_59,...,priors_11_,priors_0,priors_6_8,priors_9_10,priors_count,juv_fel_count,juv_misd_count,juv_other_count,age,id
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,0,0,1,24,4
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,69,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,34,3
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,0,1,0,23,5
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,0,0,0,43,6


In [28]:
print df_joined_reorder_po.shape
df_joined_reorder_po.to_csv('compas-r-public-order.csv', index=False)
train_po, test_po = train_test_split(df_joined_reorder_po, test_size = 0.33) 
train_po.to_csv('compas-r-public-order-train.csv', index=False)
test_po.to_csv('compas-r-public-order-test.csv', index=False)

(11003, 44)


In [29]:
df_joined_reorder_po[df_joined_reorder_po.is_po_recid == 1].shape

(625, 44)